# Feature engineering

_Goal: to predict the price of a stock in the near future._

Engineers features from raw trade. Features are stored back in the database as they can take a while to calculate.

Feature ideas:

- Summary stats of previous prices and volumes (https://alphascientist.com/feature_engineering.html).
- Trading markers (https://blog.roboforex.com/blog/2020/01/10/creating-a-trading-strategies-based-on-the-mean-reversion-and-momentum/).
- Stats on quotes.
- Relevant news articles.
- Mentions on social networks.



In [4]:
import os
import logging

import config

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from data.database import Database
from data.api_manager import API_Manager

db = Database(config.database)
api = API_Manager(config.api_key)

In [ ]:
ticker = 'MSFT'
date_from = '2015-01-01'
date_to = '2020-12-31'

ticker_details = db.get_ticker_details(ticker)
if ticker_details is None:
    ticker_details = db.store_ticker_details(api.get_ticker_details(ticker))

dates_with_trades = db.get_open_dates(ticker_details['exchange'], date_from, date_to)
dates_already_stored = db.get_stored_dates('trades', ticker)
dates_to_fetch = [d for d in dates_with_trades if d not in dates_already_stored]

logging.info(f'Fetching {len(dates_to_fetch)} days of {ticker} trades.')
for date in dates_to_fetch:
    
    time_before_fetch = time.time()
    trades = api.get_daily_trades(ticker, date)
    
    time_before_store = time.time()
    db.store_trades(ticker, date, trades)
    
    time_to_fetch = int(round(time_before_store - time_before_fetch))
    time_to_store = int(round(time.time() - time_before_store))
    logging.info(f'{ticker} {date} - fetch time: {time_to_fetch}s, store time: {time_to_store}s')